In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Import the data into a Pandas DataFrame
crime_df = pd.read_csv("output_data/crime_to_api_postal_fsa.csv")
crime_df.head()

,Index_,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,MCI,Division,Hood_ID,Neighbourhood,ObjectId,Lat,Long,Postal Code,FSA
0,159477,2018-02-18T20:45:00.000Z,2018-02-18T20:45:00.000Z,Apartment,1420,100,Assault With Weapon,Assault,D42,132,Malvern (132),146800,43.795780,-79.223686,M1B 1H5,M1B
1,21411,2018-02-25T23:45:00.000Z,2018-02-26T00:07:00.000Z,Apartment,1420,100,Assault With Weapon,Assault,D42,132,Malvern (132),19083,43.812851,-79.230248,M1B 1V8,M1B
2,89472,2018-11-10T06:30:00.000Z,2018-11-10T07:15:00.000Z,Apartment,1420,100,Assault With Weapon,Assault,D42,132,Malvern (132),82625,43.791515,-79.237824,M1B 2W1,M1B
3,96286,2018-06-09T11:30:00.000Z,2018-06-09T11:34:00.000Z,Apartment,1420,100,Assault With Weapon,Assault,D42,132,Malvern (132),88859,43.811859,-79.211212,M1B 3Z1,M1B
4,90197,2018-10-29T21:56:00.000Z,2018-10-29T21:57:00.000Z,Apartment,1420,100,Assault With Weapon,Assault,D43,134,Highland Creek (134),83421,43.792107,-79.156876,M1C 1L4,M1C


In [3]:
housing_df = pd.read_csv("Resources/Toronto_apartment_rentals_2018.csv")
housing_df.head()

,Bedroom,Bathroom,Den,Address,FSA,Lat,Long,Price
0,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,43.654155,-79.385211,$550.00
1,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,43.654155,-79.385211,$550.00
2,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,43.654155,-79.385211,$550.00
3,2,1.0,0,"Bathurst St, Toronto, ON M5S 2P9, Canada",M5S,43.658278,-79.408437,$600.00
4,1,1.0,0,", Toronto m5s1x6 ON, Canada",M5S,43.665956,-79.404799,$650.00


In [4]:
# Create a filtered dataframe from specific columns
crime_df = crime_df[['occurrencedate', 'offence', 'MCI', 'Hood_ID', 'Neighbourhood', 'Postal Code', 'FSA']].copy()
crime_df.head()

,occurrencedate,offence,MCI,Hood_ID,Neighbourhood,Postal Code,FSA
0,2018-02-18T20:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1H5,M1B
1,2018-02-25T23:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1V8,M1B
2,2018-11-10T06:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 2W1,M1B
3,2018-06-09T11:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 3Z1,M1B
4,2018-10-29T21:56:00.000Z,Assault With Weapon,Assault,134,Highland Creek (134),M1C 1L4,M1C


In [5]:
# Rename the column headers
crime_renamed = crime_df.rename(columns={"occurrencedate": "Occurrence_Date",
                                         "offence": "Offence",
                                         "MCI": "Major_Crime_Indicators",
                                         "Postal Code": "Postal_Code",
                                         "Hood_ID": "Neighbourhood_ID"})
crime_renamed.head()

,Occurrence_Date,Offence,Major_Crime_Indicators,Neighbourhood_ID,Neighbourhood,Postal_Code,FSA
0,2018-02-18T20:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1H5,M1B
1,2018-02-25T23:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1V8,M1B
2,2018-11-10T06:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 2W1,M1B
3,2018-06-09T11:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 3Z1,M1B
4,2018-10-29T21:56:00.000Z,Assault With Weapon,Assault,134,Highland Creek (134),M1C 1L4,M1C


In [6]:
# Using GroupBy in order to separate the data into fields according to "state" values
crime_fsa_df = crime_renamed.groupby(['FSA','Major_Crime_Indicators'])

# The object returned is a "GroupBy" object and cannot be viewed normally...
print(crime_fsa_df)

# In order to be visualized, a data function must be used...
crime_fsa_df.count().head(10)

Occurrence_Date  Offence  Neighbourhood_ID  \
FSA Major_Crime_Indicators                                               
M1B Assault                               4        4                 4   
M1C Assault                               2        2                 2   
M1E Assault                              17       17                17   
    Break and Enter                       4        4                 4   
M1G Assault                               9        9                 9   
    Break and Enter                       2        2                 2   
M1H Assault                               3        3                 3   
M1J Assault                              16       16                16   
    Break and Enter                       1        1                 1   
M1K Assault                               7        7                 7   

                            Neighbourhood  Postal_Code  
FSA Major_Crime_Indicators                              
M1B Assault                             4            4  
M1C Assault                             2            2  
M1E Assault                            17           17  
    Break and Enter                     4            4  
M1G Assault                             9            9  
    Break and Enter                     2            2  
M1H Assault                             3            3  
M1J Assault                            16           16  
    Break and Enter                     1            1  
M1K Assault                             7            7

In [7]:
# Create a filtered dataframe from specific columns
housing_df = housing_df[['Bedroom', 'Bathroom', 'Den', 'Address', 'FSA', 'Price']].copy()
housing_df.head()

,Bedroom,Bathroom,Den,Address,FSA,Price
0,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
1,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
2,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
3,2,1.0,0,"Bathurst St, Toronto, ON M5S 2P9, Canada",M5S,$600.00
4,1,1.0,0,", Toronto m5s1x6 ON, Canada",M5S,$650.00


In [8]:
connection_string = "root:Root123@localhost/project_db"
engine = create_engine(f'mysql://{connection_string}')

In [9]:
crime_renamed.to_sql(name='crime_data', con=engine, if_exists='append', index=True)

In [10]:
housing_df.to_sql(name='housing_data', con=engine, if_exists='append', index=True)

In [11]:
pd.read_sql_query('select * from crime_data', con=engine).head()

,index,Occurrence_Date,Offence,Major_Crime_Indicators,Neighbourhood_ID,Neighbourhood,Postal_Code,FSA
0,0,2018-02-18T20:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1H5,M1B
1,1,2018-02-25T23:45:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 1V8,M1B
2,2,2018-11-10T06:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 2W1,M1B
3,3,2018-06-09T11:30:00.000Z,Assault With Weapon,Assault,132,Malvern (132),M1B 3Z1,M1B
4,4,2018-10-29T21:56:00.000Z,Assault With Weapon,Assault,134,Highland Creek (134),M1C 1L4,M1C


In [12]:
pd.read_sql_query('select * from housing_data', con=engine).head()

,index,Bedroom,Bathroom,Den,Address,FSA,Price
0,0,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
1,1,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
2,2,1,1.0,0,"89 Chestnut St, Toronto, ON M5G 1R1, Canada",M5G,$550.00
3,3,2,1.0,0,"Bathurst St, Toronto, ON M5S 2P9, Canada",M5S,$600.00
4,4,1,1.0,0,", Toronto m5s1x6 ON, Canada",M5S,$650.00
